# Sarcasm Detection
 **Acknowledgement**

Misra, Rishabh, and Prahal Arora. "Sarcasm Detection using Hybrid Neural Network." arXiv preprint arXiv:1908.07414 (2019).

**Required Files given in below link.**

https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk

## Install `Tensorflow2.0` 

In [0]:
!!pip uninstall tensorflow
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 52kB/s 
     |████████████████████████████████| 450kB 55.1MB/s 
     |████████████████████████████████| 3.8MB 48.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=2ad2f210c79038fbead247ce95d82799de3a3f19f4a0f380b44163f4ba98e0f6
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0


In [0]:
import tensorflow
tensorflow.__version__

'2.2.0-rc2'

## Get Required Files from Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import os
os.chdir('/content/drive/My Drive/AIML_Project/NLP_Sarcasm')
!pwd

/content/drive/My Drive/AIML_Project/NLP_Sarcasm


In [0]:
#Set your project path 
project_path = '/content/drive/My Drive/AIML_Project/NLP_Sarcasm/'
images_zip_path = project_path + "Data-20200411T123902Z-002.zip"
from zipfile import ZipFile
with ZipFile(images_zip_path, 'r') as z:
  z.extractall()

In [0]:
#Set your project path 
project_path = '/content/drive/My Drive/AIML_Project/NLP_Sarcasm/'
images_zip_path = project_path + "Data-20200411T123902Z-001.zip"
from zipfile import ZipFile
with ZipFile(images_zip_path, 'r') as z:
  z.extractall()

#**## Reading and Exploring Data**

## Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get  some insights about the data. ( 4 marks)
Hint - As its in json format you need to use pandas.read_json function. Give paraemeter lines = True.

In [0]:
project_path = '/content/drive/My Drive/AIML_Project/NLP_Sarcasm/Data/'

In [4]:
import pandas as pd
dataframe = pd.read_json(project_path+'Sarcasm_Headlines_Dataset.json', lines = True)
dataframe.head(5)

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [5]:
#Exploratory Data Analysis
print("Shape of the dataframe ",dataframe.shape)
print("Sample Article_link ", dataframe['article_link'][0])
print("Sample headline ", dataframe['headline'][1])
print("Dataframe dtypes ", dataframe.dtypes)

Shape of the dataframe  (26709, 3)
Sample Article_link  https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5
Sample headline  the 'roseanne' revival catches up to our thorny political mood, for better and worse
Dataframe dtypes  article_link    object
headline        object
is_sarcastic     int64
dtype: object


## Drop `article_link` from dataset. ( 2 marks)
As we only need headline text data and is_sarcastic column for this project. We can drop artical link column here.

In [6]:
SarcasmDataframe = dataframe
SarcasmDataframe = SarcasmDataframe.drop(labels = "article_link", axis = 1)
SarcasmDataframe.head(5)

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


## Get the Length of each line and find the maximum length. ( 4 marks)
As different lines are of different length. We need to pad the our sequences using the max length.

In [7]:
print("Sample headline: ", SarcasmDataframe['headline'][26708])
SarcasmDataframe['headline'] = SarcasmDataframe['headline'].astype(str)
noOfWordsList = [len(x.split()) for x in SarcasmDataframe['headline'].tolist()]
print("Max: ",max(noOfWordsList))
print("Min: ",min(noOfWordsList))

Sample headline:  gourmet gifts for the foodie 2014
Max:  39
Min:  2


#**## Modelling**

## Import required modules required for modelling.

In [0]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D, TimeDistributed
from tensorflow.keras.models import Model, Sequential

# Set Different Parameters for the model. ( 2 marks)

In [0]:
max_features = 10000
maxlen = 39
embedding_size = 200

## Apply Keras Tokenizer of headline column of your data.  ( 4 marks)
Hint - First create a tokenizer instance using Tokenizer(num_words=max_features) 
And then fit this tokenizer instance on your data column df['headline'] using .fit_on_texts()

In [0]:
tokenizer = Tokenizer(num_words=max_features,filters= '!"#$%&()*+,-./:;<=>?@[\]^_`{|}\n“~')
tokenizer.fit_on_texts(SarcasmDataframe['headline'])

# Define X and y for your model.

In [11]:
X = tokenizer.texts_to_sequences(SarcasmDataframe['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(SarcasmDataframe['is_sarcastic'])

print("Number of Samples:", len(X))
print(X[0])
print("Number of Labels: ", len(y))
print(y[0])

Number of Samples: 26709
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0  307  678 3336 2297   47  381 2575    5 2576 8433]
Number of Labels:  26709
0


## Get the Vocabulary size ( 2 marks)
Hint : You can use tokenizer.word_index.

In [12]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

29652


#**## Word Embedding**

## Get Glove Word Embeddings

In [0]:
glove_file = project_path + "glove.6B.zip"

In [0]:
#Extract Glove embedding zip file
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

# Get the Word Embeddings using Embedding file as given below.

In [0]:
EMBEDDING_FILE = './glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd



# Create a weight matrix for words in training docs

In [16]:
num_words = vocab_size
embedding_matrix = np.zeros((num_words, 200))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

## Create and Compile your Model  ( 7 marks)
Hint - Use Sequential model instance and then add Embedding layer, Bidirectional(LSTM) layer, then dense and dropout layers as required. 
In the end add a final dense layer with sigmoid activation for binary classification.


In [17]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential([
  layers.Embedding(vocab_size, embedding_size, weights = [embedding_matrix]),
  layers.Bidirectional(LSTM(128, return_sequences=True)),
  layers.Dropout(0.3),
  layers.Dense(100),
  layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 200)         5930400   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 256)         336896    
_________________________________________________________________
dropout (Dropout)            (None, None, 256)         0         
_________________________________________________________________
dense (Dense)                (None, None, 100)         25700     
_________________________________________________________________
dense_1 (Dense)              (None, None, 1)           101       
Total params: 6,293,097
Trainable params: 6,293,097
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

# Fit your model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy ( 5 marks)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [20]:
print("X_train.shape ", X_train.shape)
print("X_test.shape ", X_test.shape)
print("y_train.shape ", y_train.shape)
print("y_test.shape ", y_test.shape)

X_train.shape  (21367, 39)
X_test.shape  (5342, 39)
y_train.shape  (21367,)
y_test.shape  (5342,)


In [21]:
batch_size = 100
epochs = 5

## Add your code here ##
history = model.fit(X_train, y_train, batch_size=100, epochs=5, validation_split=0.2, verbose=1) # Train

Epoch 1/5
171/171 [==============================] - 11s 66ms/step - loss: 0.4590 - accuracy: 0.7770 - val_loss: 0.3682 - val_accuracy: 0.8449
Epoch 2/5
171/171 [==============================] - 11s 62ms/step - loss: 0.2672 - accuracy: 0.8898 - val_loss: 0.3372 - val_accuracy: 0.8565
Epoch 3/5
171/171 [==============================] - 11s 62ms/step - loss: 0.1864 - accuracy: 0.9290 - val_loss: 0.3482 - val_accuracy: 0.8561
Epoch 4/5
171/171 [==============================] - 11s 63ms/step - loss: 0.1185 - accuracy: 0.9565 - val_loss: 0.4394 - val_accuracy: 0.8490
Epoch 5/5
171/171 [==============================] - 11s 62ms/step - loss: 0.0763 - accuracy: 0.9734 - val_loss: 0.5253 - val_accuracy: 0.8567


Validation Accuracy for the above 5 Epochs = 0.85264

# Predict Test Accuracy

In [22]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Model Accuracy is ",test_acc)

167/167 [==============================] - 1s 5ms/step - loss: 0.4908 - accuracy: 0.8655
Model Accuracy is  0.8655405044555664
